In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import locale  # to get french time
import pandas as pd
import time
import sqlite3  # for SQL conversion

In [17]:
resto_url = 'https://www.crous-strasbourg.fr/restaurant/cafeteria-le-pege-2/'

In [33]:
# Function to scrape the full menu of a restaurant
def scrape_full_menu(resto_url):
    menuInfo = {"day": [], "lunch": [], "dinner": []}

    driver = webdriver.Firefox()

    # Access the restaurant's page
    driver.get(resto_url)

    # Wait for the page to load the "HORAIRES" element in the info section
    element = WebDriverWait(driver, 10).until(
        EC.text_to_be_present_in_element((By.CSS_SELECTOR, 'div.info'), "HORAIRES")
    )

    try:
        # Try to reject cookies
        reject_all = driver.find_element(By.XPATH, "/html/body/div[6]/div/div[2]/div/div[2]/button[1]")
        reject_all.click()
    except Exception:
        try:
            reject_all = driver.find_element(By.ID, "tru_deselect_btn")
            reject_all.click()
        except Exception:
            pass  # Ignore if the reject window is not found

    # Scrape all menu information
    driver.execute_script("window.scrollBy(0, 300);")
    time.sleep(0.5)
    date_elements = driver.find_elements(By.CSS_SELECTOR, 'time.menu_date_title')

    meal_div = driver.find_elements(By.CSS_SELECTOR, 'div.meal')
    meals = []
    for meal in meal_div:
        if meal.text != "":
            meals.append(meal.text)

    # Extract the scraped day
    scraped_day = ' '.join(date_elements[0].text.split()[2:5])
    menuInfo["day"].append(scraped_day)

    # Extract the meals (lunch and dinner)
       # Extract the meals (lunch and dinner)
    if len(meals) == 2:
        lunch = meals[0].replace('\n', ' ')  # Ensure correct format for lunch
        dinner = meals[1].replace('\n', ' ')  # Ensure correct format for dinner
    elif len(meals) == 1:
        lunch = meals[0].replace('\n', ' ')
        dinner = ""
    else:
        lunch = ""
        dinner = ""

    # Append lunch and dinner data with line breaks
    menuInfo["lunch"].append(lunch)
    menuInfo["dinner"].append(dinner)

    driver.quit()

    # Convert the results into a DataFrame
    menuInfo_df = pd.DataFrame(menuInfo)

    # Return the DataFrame
    return menuInfo_df

# Function to filter the menu by a specific day and meal
def filter_menu(menuInfo_df, specific_day, specific_meal):
    filtered_menu = menuInfo_df[menuInfo_df['day'] == specific_day]

    if specific_meal == "lunch":
        return filtered_menu[['day', 'lunch']]
    elif specific_meal == "dinner":
        return filtered_menu[['day', 'dinner']]
    else:
        return filtered_menu

In [35]:
# Set french localisation for month and day of the week
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Function to get today's date in the format used by the menu (e.g., "jeudi 10 octobre")
def get_today_date():
    # Ottieni la data odierna
    today = datetime.today()
    # Formatta la data nel formato "jeudi 10 octobre"
    formatted_date = today.strftime("%A %d %B")
    return formatted_date

# Function to filter the menu by a specific day and meal
def filter_menu(menuInfo_df, specific_day, specific_meal):
    filtered_menu = menuInfo_df[menuInfo_df['day'] == specific_day]

    if specific_meal == "lunch":
        return filtered_menu[['day', 'lunch']]
    elif specific_meal == "dinner":
        return filtered_menu[['day', 'dinner']]
    else:
        return filtered_menu

# Example usage: Scraping the full menu of a single restaurant and filtering it
resto_url = "https://www.crous-strasbourg.fr/restaurant/cafeteria-le-pege-2/"  # Restaurant URL
specific_day = get_today_date()  # Automatically get today's date
print(specific_day)
specific_meal = "lunch"  # Specific meal ("lunch" or "dinner")

# Call the function to scrape the full menu of the restaurant
menuInfo_df = scrape_full_menu(resto_url)

# Display the full DataFrame (all days and meals scraped from the page)
print("Full menu DataFrame:")
print(menuInfo_df)

# Filter the menu by the specific day and meal
filtered_menu = filter_menu(menuInfo_df, specific_day, specific_meal)

# Display the filtered result
print(f"\nFiltered menu for {specific_day} ({specific_meal}):")
print(filtered_menu)

mercredi 16 octobre
Full menu DataFrame:
                   day                                              lunch  \
0  mercredi 16 octobre  Déjeuner Petit-déjeuner Café & Viennoiseries 7...   

  dinner  
0         

Filtered menu for mercredi 16 octobre (lunch):
                   day                                              lunch
0  mercredi 16 octobre  Déjeuner Petit-déjeuner Café & Viennoiseries 7...


In [37]:
print(menuInfo_df['lunch'][0])

Déjeuner Petit-déjeuner Café & Viennoiseries 7h30 -10h30 Plat du jour Omelette au fromage Ratatouille Gratin de pommes de terre ou Frites Végétarien Cappelletis 5 fromages, sauce crème Roquette & parmesan Origine de nos viandes du jour FRANCE


In [50]:
# Convert the dataframe into SQL file
# Connect to an SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('menu.db')

# Convert the DataFrame to SQL
menuInfo_df.to_sql('menu', conn, if_exists='replace', index=False)

# Verify the insertion by reading back the data
df_from_sql = pd.read_sql_query("SELECT * FROM menu", conn)
print(df_from_sql)

# Close the connection
conn.close()

                   day                                              lunch  \
0  mercredi 16 octobre  Déjeuner Petit-déjeuner Café & Viennoiseries 7...   

  dinner  
0         
